# Exploratory Analysis of Loan Data from Prosper

## Preliminary Wrangling

This notebook explores a dataset containing 113,937 loans with 81 fields on each loan.

In [1]:
# importing all necessary packages and setting plots to be embedded inline
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sb

%matplotlib inline

> Loading the data into a dataframe

In [2]:
# reading the CSV and loading the data into a dataframe
loan_data = pd.read_csv('prosperLoanData.csv', index_col=None)
loan_data.head()

,ListingKey,ListingNumber,ListingCreationDate,CreditGrade,Term,LoanStatus,ClosedDate,BorrowerAPR,BorrowerRate,LenderYield,...,LP_ServiceFees,LP_CollectionFees,LP_GrossPrincipalLoss,LP_NetPrincipalLoss,LP_NonPrincipalRecoverypayments,PercentFunded,Recommendations,InvestmentFromFriendsCount,InvestmentFromFriendsAmount,Investors
0,1021339766868145413AB3B,193129,2007-08-26 19:09:29.263000000,C,36,Completed,2009-08-14 00:00:00,0.16516,0.1580,0.1380,...,-133.18,0.0,0.0,0.0,0.0,1.0,0,0,0.0,258
1,10273602499503308B223C1,1209647,2014-02-27 08:28:07.900000000,NaN,36,Current,NaN,0.12016,0.0920,0.0820,...,0.00,0.0,0.0,0.0,0.0,1.0,0,0,0.0,1
2,0EE9337825851032864889A,81716,2007-01-05 15:00:47.090000000,HR,36,Completed,2009-12-17 00:00:00,0.28269,0.2750,0.2400,...,-24.20,0.0,0.0,0.0,0.0,1.0,0,0,0.0,41
3,0EF5356002482715299901A,658116,2012-10-22 11:02:35.010000000,NaN,36,Current,NaN,0.12528,0.0974,0.0874,...,-108.01,0.0,0.0,0.0,0.0,1.0,0,0,0.0,158
4,0F023589499656230C5E3E2,909464,2013-09-14 18:38:39.097000000,NaN,36,Current,NaN,0.24614,0.2085,0.1985,...,-60.27,0.0,0.0,0.0,0.0,1.0,0,0,0.0,20


In [3]:
# getting the shape of the dataset
loan_data.shape

(113937, 81)

There are 81 fields available for each of the loan records. We will focus on only a few (20 fields) of these fields for further exploratory analysis. So, we can remove the rest of the fields from the dataset.

In [4]:
# retaining only the required columns in the dataframe
loan_data = loan_data[['LoanNumber', 'LoanOriginalAmount', 'LoanOriginationDate', 'ListingNumber', 'Term', 'LoanStatus', 'BorrowerRate', 'ProsperScore', 'ListingCategory (numeric)', 'BorrowerState', 'Occupation', 'EmploymentStatus', 'IsBorrowerHomeowner', 'CreditScoreRangeLower', 'CreditScoreRangeUpper', 'IncomeRange', 'TotalProsperLoans', 'TotalProsperPaymentsBilled', 'OnTimeProsperPayments', 'Recommendations']]
loan_data.shape

(113937, 20)

In [5]:
# renaming the columns
loan_data.columns = ['loan_number', 'loan_original_amount', 'loan_origination_date', 'listing_number', 'term', 'loan_status', 
                     'borrower_rate', 'prosper_score', 'listing_category', 'borrower_state', 'occupation', 'employment_status', 
                     'is_borrower_homeowner', 'credit_score_range_lower', 'credit_score_range_upper', 'income_range', 
                     'total_prosper_loans', 'total_prosper_payments_billed', 'on_time_prosper_payments', 'recommendations']
loan_data.head()

,loan_number,loan_original_amount,loan_origination_date,listing_number,term,loan_status,borrower_rate,prosper_score,listing_category,borrower_state,occupation,employment_status,is_borrower_homeowner,credit_score_range_lower,credit_score_range_upper,income_range,total_prosper_loans,total_prosper_payments_billed,on_time_prosper_payments,recommendations
0,19141,9425,2007-09-12 00:00:00,193129,36,Completed,0.1580,NaN,0,CO,Other,Self-employed,True,640.0,659.0,"$25,000-49,999",NaN,NaN,NaN,0
1,134815,10000,2014-03-03 00:00:00,1209647,36,Current,0.0920,7.0,2,CO,Professional,Employed,False,680.0,699.0,"$50,000-74,999",NaN,NaN,NaN,0
2,6466,3001,2007-01-17 00:00:00,81716,36,Completed,0.2750,NaN,0,GA,Other,Not available,False,480.0,499.0,Not displayed,NaN,NaN,NaN,0
3,77296,10000,2012-11-01 00:00:00,658116,36,Current,0.0974,9.0,16,GA,Skilled Labor,Employed,True,800.0,819.0,"$25,000-49,999",NaN,NaN,NaN,0
4,102670,15000,2013-09-20 00:00:00,909464,36,Current,0.2085,4.0,2,MN,Executive,Employed,True,680.0,699.0,"$100,000+",1.0,11.0,11.0,0


In [6]:
# getting information regarding data type and missing values for each field
loan_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 113937 entries, 0 to 113936
Data columns (total 20 columns):
 #   Column                         Non-Null Count   Dtype  
---  ------                         --------------   -----  
 0   loan_number                    113937 non-null  int64  
 1   loan_original_amount           113937 non-null  int64  
 2   loan_origination_date          113937 non-null  object 
 3   listing_number                 113937 non-null  int64  
 4   term                           113937 non-null  int64  
 5   loan_status                    113937 non-null  object 
 6   borrower_rate                  113937 non-null  float64
 7   prosper_score                  84853 non-null   float64
 8   listing_category               113937 non-null  int64  
 9   borrower_state                 108422 non-null  object 
 10  occupation                     110349 non-null  object 
 11  employment_status              111682 non-null  object 
 12  is_borrower_homeowner         

> Handling the missing values and checking data types of fields in the dataset

We observe that the following fields have missing values - `prosper_score`, `borrower_state`, `occupation`, `employment_status`, `credit_score_range_lower`, `credit_score_range_upper`, `total_prosper_loans`, `total_prosper_payments_billed` and  `on_time_prosper_payments`.

The data types of the fields should also be checked before proceeding with the exploratory analysis. `loan_origination_date` should be of datetime format rather than string. `employment_status`, `income_range` and `loan_status` should be categorical fields rather than strings.

(1) Previous loan fields (`total_prosper_loans`, `total_prosper_payments_billed` and  `on_time_prosper_payments`)

In [7]:
# getting the unique values of total number of loans
loan_data.total_prosper_loans.unique()

array([nan,  1.,  3.,  2.,  4.,  6.,  5.,  8.,  7.,  0.])

In [8]:
# replacing null values with 0
loan_data.total_prosper_loans.fillna(0, inplace=True)

In [9]:
# converting the field to integer data type since the values can never have decimal values
loan_data.total_prosper_loans = loan_data.total_prosper_loans.astype(np.int64)
# checking if the conversion is successful
loan_data.total_prosper_loans.dtype

dtype('int64')

In [10]:
# getting the unique values of total number of loan payments made previously
loan_data.total_prosper_payments_billed.unique()

array([ nan,  11.,  67.,  12.,   8.,   9.,  20.,   1.,   4.,  16.,   6.,
        10.,  32.,  35.,  14.,  29.,   5.,  22.,  13.,  24.,   3.,  25.,
        15.,  46.,   2.,  30.,  27.,  18.,  28.,  26.,  34.,  74.,  36.,
        54.,  63.,   7.,  50.,  33.,  21.,  19.,  38.,  42.,  39.,  65.,
        23.,  95.,  56.,  53.,  66.,  51.,  76.,  17.,  48.,  78., 107.,
        77., 131.,  58.,  44., 111.,  61.,  31.,  55.,  59.,  52., 110.,
        62.,  47.,  37.,  69.,  73.,  57.,  64.,  89.,  72., 101.,  83.,
       103.,  70.,  40., 102.,  49.,  60.,  41.,  45.,  43.,  75.,   0.,
        68.,  81., 118.,  71.,  90.,  88.,  82.,  98.,  80., 112., 104.,
        99.,  86.,  79.,  96., 124.,  85., 105.,  87.,  91.,  93.,  84.,
       109.,  94., 100., 123.,  97.,  92., 116., 119., 132., 106., 128.,
       117., 115., 122., 134., 120., 125., 133., 121., 113., 141.])

In [11]:
# replacing null values with 0
loan_data.total_prosper_payments_billed.fillna(0, inplace=True)

In [12]:
# converting the field to integer data type since the values can never have decimal values
loan_data.total_prosper_payments_billed = loan_data.total_prosper_payments_billed.astype(np.int64)
# checking if the conversion is successful
loan_data.total_prosper_payments_billed.dtype

dtype('int64')

In [13]:
# getting the unique values of total number of timely loan payments made previously
loan_data.on_time_prosper_payments.unique()

array([ nan,  11.,  67.,  12.,   8.,   9.,  20.,   1.,   4.,  16.,   6.,
        10.,  32.,  35.,  14.,   7.,  28.,   5.,  22.,  13.,  24.,   3.,
        25.,  15.,  46.,  30.,  31.,   2.,  27.,  18.,  26.,  34.,  74.,
        54.,  33.,  63.,  50.,  21.,  19.,  38.,  42.,  39.,  61.,  23.,
        95.,  56.,  53.,  48.,  66.,   0.,  76.,  29.,  51.,  78.,  17.,
        52., 105.,  57., 131.,  36., 111.,  58.,  77.,  55.,  59., 110.,
        47.,  64.,  37.,  69.,  45.,  70.,  89.,  65.,  60.,  72.,  44.,
       101.,  83.,  62., 103.,  92., 102.,  43.,  73.,  49.,  41.,  40.,
        75.,  68.,  81., 118.,  71.,  88.,  80.,  96.,  97.,  84.,  79.,
       124.,  85.,  90., 104.,  82.,  87.,  91.,  99., 100.,  93.,  94.,
       116., 119., 117., 132., 106.,  86., 128., 114., 122., 134., 107.,
       123., 125., 133., 121., 115., 112., 141.])

In [14]:
# replacing null values with 0
loan_data.on_time_prosper_payments.fillna(0, inplace=True)

In [15]:
# converting the field to integer data type since the values can never have decimal values
loan_data.on_time_prosper_payments = loan_data.total_prosper_payments_billed.astype(np.int64)
# checking if the conversion is successful
loan_data.on_time_prosper_payments.dtype

dtype('int64')

(2) Risk score (`prosper_score`)

In [16]:
# getting the unique values of prosper score
loan_data.prosper_score.unique()

array([nan,  7.,  9.,  4., 10.,  2., 11.,  8.,  5.,  3.,  6.,  1.])

In [17]:
# replacing null values with 0 (prosper score not available)
loan_data.prosper_score.fillna(0, inplace=True)

In [18]:
# converting the field to integer data type since the values never have decimal values
loan_data.prosper_score = loan_data.prosper_score.astype(np.int64)
# checking if the conversion is successful
loan_data.prosper_score.dtype

dtype('int64')

(3) Borrower's State (`borrower_state`)

In [19]:
# getting the unique values of borrower's state
loan_data.borrower_state.unique()

array(['CO', 'GA', 'MN', 'NM', 'KS', 'CA', 'IL', 'MD', nan, 'AL', 'AZ',
       'VA', 'FL', 'PA', 'OR', 'MI', 'NY', 'LA', 'WI', 'OH', 'NC', 'WA',
       'NV', 'NJ', 'TX', 'AR', 'SC', 'DE', 'MO', 'NE', 'UT', 'DC', 'MA',
       'CT', 'IN', 'KY', 'OK', 'MS', 'WV', 'RI', 'TN', 'ID', 'MT', 'HI',
       'NH', 'VT', 'WY', 'ME', 'AK', 'IA', 'SD', 'ND'], dtype=object)

In [20]:
# replacing null values with 'Not Available' (state not available)
loan_data.borrower_state.fillna('Not Available', inplace=True)

(4) `occupation` and `employment_status`

In [21]:
# getting the unique values of occupation
loan_data.occupation.unique()

array(['Other', 'Professional', 'Skilled Labor', 'Executive',
       'Sales - Retail', 'Laborer', 'Food Service', 'Fireman',
       'Waiter/Waitress', 'Construction', 'Computer Programmer',
       'Sales - Commission', 'Retail Management', 'Engineer - Mechanical',
       'Military Enlisted', 'Clerical', nan, 'Teacher', 'Clergy',
       'Accountant/CPA', 'Attorney', 'Nurse (RN)', 'Analyst',
       "Nurse's Aide", 'Investor', 'Realtor', 'Flight Attendant',
       'Nurse (LPN)', 'Military Officer', 'Food Service Management',
       'Truck Driver', 'Administrative Assistant',
       'Police Officer/Correction Officer', 'Social Worker',
       'Tradesman - Mechanic', 'Medical Technician', 'Professor',
       'Postal Service', 'Civil Service', 'Pharmacist',
       'Tradesman - Electrician', 'Scientist', 'Dentist',
       'Engineer - Electrical', 'Architect', 'Landscaping',
       'Tradesman - Carpenter', 'Bus Driver', 'Tradesman - Plumber',
       'Engineer - Chemical', 'Doctor', 'Chemist',


In [22]:
# replacing null values with 'Not Available' (occupation not available)
loan_data.occupation.fillna('Not Available', inplace=True)

In [23]:
# getting the unique values of employment status
loan_data.employment_status.unique()

array(['Self-employed', 'Employed', 'Not available', 'Full-time', 'Other',
       nan, 'Not employed', 'Part-time', 'Retired'], dtype=object)

In [24]:
# replacing null values with 'Not Available' (employment status not available)
loan_data.employment_status.fillna('Not Available', inplace=True)

In [25]:
# converting employment status to category data type
loan_data.employment_status = loan_data.employment_status.astype('category')

(5) `credit_score_range_lower` and `credit_score_range_upper`

In [26]:
# getting the unique values of lower range of employment score
loan_data.credit_score_range_lower.unique()

array([640., 680., 480., 800., 740., 700., 820., 760., 660., 620., 720.,
       520., 780., 600., 580., 540., 560., 500., 840., 860.,  nan, 460.,
         0., 880., 440., 420., 360.])

In [27]:
# getting the unique values of upper range of employment score
loan_data.credit_score_range_upper.unique()

array([659., 699., 499., 819., 759., 719., 839., 779., 679., 639., 739.,
       539., 799., 619., 599., 559., 579., 519., 859., 879.,  nan, 479.,
        19., 899., 459., 439., 379.])

In [28]:
# replacing null values with 0 wherever the lower/upper score range is not available
loan_data.credit_score_range_lower.fillna(0, inplace=True)
loan_data.credit_score_range_upper.fillna(0, inplace=True)

In [29]:
# converting the fields to integer data type since there are no decimal values
loan_data.credit_score_range_lower = loan_data.credit_score_range_lower.astype(np.int64)
loan_data.credit_score_range_upper = loan_data.credit_score_range_upper.astype(np.int64)

(6) `loan_origination_date`

In [30]:
# converting loan_origination_date to datetime format
loan_data.loan_origination_date = pd.to_datetime(loan_data.loan_origination_date)
# checking if the convertion is successful
loan_data.loan_origination_date.dtype

dtype('<M8[ns]')

(7) `loan_status` 

In [31]:
# getting the unique values of loan status
loan_data.loan_status.unique()

array(['Completed', 'Current', 'Past Due (1-15 days)', 'Defaulted',
       'Chargedoff', 'Past Due (16-30 days)', 'Cancelled',
       'Past Due (61-90 days)', 'Past Due (31-60 days)',
       'Past Due (91-120 days)', 'FinalPaymentInProgress',
       'Past Due (>120 days)'], dtype=object)

In [32]:
# converting loan status to category data type
loan_data.loan_status = loan_data.loan_status.astype('category')

(8) `income_range`

In [33]:
# getting the unique values of income range
loan_data.income_range.unique()

array(['$25,000-49,999', '$50,000-74,999', 'Not displayed', '$100,000+',
       '$75,000-99,999', '$1-24,999', 'Not employed', '$0'], dtype=object)

In [34]:
# ordered list of categories 
ordered_categories = ['Not displayed', 'Not employed', '$0', '$1-24,999', '$25,000-49,999', '$50,000-74,999', 
                      '$75,000-99,999', '$100,000+']
# converting to ordered categorical field
ordered_var = pd.api.types.CategoricalDtype(ordered = True, categories = ordered_categories)
loan_data.income_range = loan_data.income_range.astype(ordered_var)

In [35]:
# converting income range to category data type
loan_data.income_range = loan_data.income_range.astype('category')

> Verification

In [36]:
# checking missing values and data type for each field
loan_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 113937 entries, 0 to 113936
Data columns (total 20 columns):
 #   Column                         Non-Null Count   Dtype         
---  ------                         --------------   -----         
 0   loan_number                    113937 non-null  int64         
 1   loan_original_amount           113937 non-null  int64         
 2   loan_origination_date          113937 non-null  datetime64[ns]
 3   listing_number                 113937 non-null  int64         
 4   term                           113937 non-null  int64         
 5   loan_status                    113937 non-null  category      
 6   borrower_rate                  113937 non-null  float64       
 7   prosper_score                  113937 non-null  int64         
 8   listing_category               113937 non-null  int64         
 9   borrower_state                 113937 non-null  object        
 10  occupation                     113937 non-null  object        
 11  

> There are now, no missing values in the dataframe.

In [37]:
# getting the quartile values and other statistics for the numerical fields
loan_data.describe()

,loan_number,loan_original_amount,listing_number,term,borrower_rate,prosper_score,listing_category,credit_score_range_lower,credit_score_range_upper,total_prosper_loans,total_prosper_payments_billed,on_time_prosper_payments,recommendations
count,113937.000000,113937.00000,1.139370e+05,113937.000000,113937.000000,113937.00000,113937.000000,113937.000000,113937.000000,113937.000000,113937.000000,113937.000000,113937.000000
mean,69444.474271,8337.01385,6.278857e+05,40.830248,0.192764,4.43123,2.774209,682.011638,700.913084,0.275459,4.445483,4.445483,0.048027
std,38930.479610,6245.80058,3.280762e+05,10.436212,0.074818,3.30703,3.996797,82.577903,83.399069,0.654774,12.410277,12.410277,0.332353
min,1.000000,1000.00000,4.000000e+00,12.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,37332.000000,4000.00000,4.009190e+05,36.000000,0.134000,0.00000,1.000000,660.000000,679.000000,0.000000,0.000000,0.000000,0.000000
50%,68599.000000,6500.00000,6.005540e+05,36.000000,0.184000,5.00000,1.000000,680.000000,699.000000,0.000000,0.000000,0.000000,0.000000
75%,101901.000000,12000.00000,8.926340e+05,36.000000,0.250000,7.00000,3.000000,720.000000,739.000000,0.000000,0.000000,0.000000,0.000000
max,136486.000000,35000.00000,1.255725e+06,60.000000,0.497500,11.00000,20.000000,880.000000,899.000000,8.000000,141.000000,141.000000,39.000000


### What is the structure of your dataset?

> There are 113,937 loans in the dataset with 81 features out of which we have retained 20 features in order to limit the scope of the analysis. Most variables are numeric in nature, but the variables `loan_status`, `employment_status` and `income_range` are categorical in nature. Further, `income_range` is ordered.

### What is/are the main feature(s) of interest in your dataset?

> The features of interest are those that are useful to predict the status of the loan.

### What features in the dataset do you think will help support your investigation into your feature(s) of interest?

> I expect the employment status of the borrower, income range, details of previous loans and credit score to play a major role in the analysis.

## Univariate Exploration

> In this section, investigate distributions of individual variables. If
you see unusual points or outliers, take a deeper look to clean things up
and prepare yourself to look at relationships between variables.

> Make sure that, after every plot or related series of plots, that you
include a Markdown cell with comments about what you observed, and what
you plan on investigating next.

### Discuss the distribution(s) of your variable(s) of interest. Were there any unusual points? Did you need to perform any transformations?

> Your answer here!

### Of the features you investigated, were there any unusual distributions? Did you perform any operations on the data to tidy, adjust, or change the form of the data? If so, why did you do this?

> Your answer here!

## Bivariate Exploration

> In this section, investigate relationships between pairs of variables in your
data. Make sure the variables that you cover here have been introduced in some
fashion in the previous section (univariate exploration).

### Talk about some of the relationships you observed in this part of the investigation. How did the feature(s) of interest vary with other features in the dataset?

> Your answer here!

### Did you observe any interesting relationships between the other features (not the main feature(s) of interest)?

> Your answer here!

## Multivariate Exploration

> Create plots of three or more variables to investigate your data even
further. Make sure that your investigations are justified, and follow from
your work in the previous sections.

### Talk about some of the relationships you observed in this part of the investigation. Were there features that strengthened each other in terms of looking at your feature(s) of interest?

> Your answer here!

### Were there any interesting or surprising interactions between features?

> Your answer here!

> At the end of your report, make sure that you export the notebook as an
html file from the `File > Download as... > HTML` menu. Make sure you keep
track of where the exported file goes, so you can put it in the same folder
as this notebook for project submission. Also, make sure you remove all of
the quote-formatted guide notes like this one before you finish your report!